In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [1]:
import os
project_folder = '/gdrive/MyDrive/ProjectCIRI'
my_module_path = os.path.join(project_folder, 'code', 'ciri_utils')

In [33]:
!pip install $my_module_path

Processing /gdrive/MyDrive/ProjectCIRI/code/ciri_utils
  Preparing metadata (setup.py) ... done
  Created wheel for ciri-utils: filename=ciri_utils-0.1-py3-none-any.whl size=25819 sha256=b1a88a76c005fbdcf4c42c489caf4ec494a0522a25262c452c909960b3adaa9e
  Stored in directory: /tmp/pip-ephem-wheel-cache-5t1mltbs/wheels/af/b2/73/860a75cbbf27b28ca0320a77f4feeba042b17de09092a60e68
Successfully built ciri-utils
  Attempting uninstall: ciri-utils
    Found existing installation: ciri-utils 0.1
    Uninstalling ciri-utils-0.1:
      Successfully uninstalled ciri-utils-0.1


In [2]:
from ciri_utils.engine_v2 import CIRI_trainer
import ray
from ray import tune

In [5]:
selected_model="efficientnet_b0"
root_base = os.path.join(project_folder, 'Incidents-subset')
root_augmented = os.path.join(project_folder, 'augmented_images')

data_folders=[root_base, root_augmented]

In [6]:
ciri_trainer = CIRI_trainer(model=selected_model,
                            data_folders=data_folders,
                            data_prop=0.8,
                            sample_indices=list(range(1000)), # remove this for full dataset
                            )

In [13]:
param_space = {
    "epochs": 1,
    "batch_size": tune.choice([16, 32, 64]),
    "lr": tune.loguniform(1e-4, 1e-2)
}
# ciri_trainable = ciri_trainer.get_trainer(run_name="test_tuning",
#                                      as_trainable=True)

In [12]:
!rm -rf /root/ray_results/tune_test

In [14]:
results = ciri_trainer.tune_hyperparams(
	run_name="tune_test",
	param_space=param_space,
	num_samples=2
)

2024-04-13 08:01:46,865	INFO tune.py:622 -- [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949


Defaulting to ASHA scheduler (no scheduler provided or not an instance of TrialScheduler)
+----------------------------------------------------------+
| Configuration for experiment     tune_test               |
+----------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator   |
| Scheduler                        AsyncHyperBandScheduler |
| Number of trials                 2                       |
+----------------------------------------------------------+

View detailed results here: /root/ray_results/tune_test
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-04-13_07-57-24_347913_579/artifacts/2024-04-13_08-01-46/tune_test/driver_artifacts`

Trial status: 2 PENDING
Current time: 2024-04-13 08:01:47. Total running time: 0s
Logical resource usage: 0/2 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:T4)
+---------------------------------------------------------------------------------------+
|

(TrainTrainable pid=4752) 2024-04-13 08:01:53.532880: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(TrainTrainable pid=4752) 2024-04-13 08:01:53.532942: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(TrainTrainable pid=4752) 2024-04-13 08:01:53.535312: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(TrainTrainable pid=4752) 2024-04-13 08:01:55.259365: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



Trial TorchTrainer_132f3_00000 started with configuration:
+------------------------------------------------------+
| Trial TorchTrainer_132f3_00000 config                |
+------------------------------------------------------+
| train_loop_config/batch_size                      32 |
| train_loop_config/epochs                           1 |
| train_loop_config/lr                      0.00013785 |
+------------------------------------------------------+


(TorchTrainer pid=4752) Started distributed worker processes: 
(TorchTrainer pid=4752) - (ip=172.28.0.12, pid=4824) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=4824) Setting up process group for: env:// [rank=0, world_size=1]
(RayTrainWorker pid=4824) [W Utils.hpp:133] Warning: Environment variable NCCL_ASYNC_ERROR_HANDLING is deprecated; use TORCH_NCCL_ASYNC_ERROR_HANDLING instead (function getCvarInt)
(RayTrainWorker pid=4824) 2024-04-13 08:02:00.614581: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(RayTrainWorker pid=4824) 2024-04-13 08:02:00.614634: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(RayTrainWorker pid=4824) 2024-04-13 08:02:00.615827: E external/local_xla/xla/stream_executor/cuda/cuda

...


Trial TorchTrainer_132f3_00000 completed after 1 iterations at 2024-04-13 08:02:57. Total running time: 1min 10s
+---------------------------------------------------------------+
| Trial TorchTrainer_132f3_00000 result                         |
+---------------------------------------------------------------+
| checkpoint_dir_name                         checkpoint_000000 |
| time_this_iter_s                                     60.61207 |
| time_total_s                                         60.61207 |
| training_iteration                                          1 |
| accuracy                                               0.8995 |
| loss                                                  2.01431 |
| summary/epoch/0                                           1.0 |
| summary/train_acc/0                       0.21401752190237797 |
| summary/train_loss/0                       3.2314441479169407 |
| summary/val_acc/0                          0.8994974874371859 |
| summary/val_loss/0        

(TorchTrainer pid=4752) Started distributed worker processes: 
(TorchTrainer pid=4752) - (ip=172.28.0.12, pid=5139) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=5139) Setting up process group for: env:// [rank=0, world_size=1]
(RayTrainWorker pid=5139) [W Utils.hpp:133] Warning: Environment variable NCCL_ASYNC_ERROR_HANDLING is deprecated; use TORCH_NCCL_ASYNC_ERROR_HANDLING instead (function getCvarInt)
(RayTrainWorker pid=5139) 2024-04-13 08:03:04.294434: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(RayTrainWorker pid=5139) 2024-04-13 08:03:04.294512: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(RayTrainWorker pid=5139) 2024-04-13 08:03:04.296134: E external/local_xla/xla/stream_executor/cuda/cuda


Trial status: 1 TERMINATED | 1 RUNNING
Current time: 2024-04-13 08:03:17. Total running time: 1min 30s
Logical resource usage: 0/2 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:T4)
+--------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name                 status         ...config/batch_size     train_loop_config/lr     iter     total time (s)      loss     accuracy |
+--------------------------------------------------------------------------------------------------------------------------------------------+
| TorchTrainer_132f3_00001   RUNNING                          64              0.000251265                                                    |
| TorchTrainer_132f3_00000   TERMINATED                       32              0.00013785         1            60.6121   2.01431     0.899497 |
+---------------------------------------------------------------------------------------------------------

Epoch (test) 1: 100%|██████████| 7/7 [00:05<00:00,  1.21it/s]
(RayTrainWorker pid=5139) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/tune_test/TorchTrainer_132f3_00001_1_batch_size=64,lr=0.0003_2024-04-13_08-01-46/checkpoint_000000)
2024-04-13 08:03:38,460	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).
2024-04-13 08:03:38,463	INFO tune.py:1016 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/tun


Trial TorchTrainer_132f3_00001 completed after 1 iterations at 2024-04-13 08:03:38. Total running time: 1min 51s
+---------------------------------------------------------------+
| Trial TorchTrainer_132f3_00001 result                         |
+---------------------------------------------------------------+
| checkpoint_dir_name                         checkpoint_000000 |
| time_this_iter_s                                     39.79741 |
| time_total_s                                         39.79741 |
| training_iteration                                          1 |
| accuracy                                               0.8995 |
| loss                                                  2.06753 |
| summary/epoch/0                                           1.0 |
| summary/train_acc/0                       0.36795994993742176 |
| summary/train_loss/0                        2.446434795856476 |
| summary/val_acc/0                          0.8994974874371859 |
| summary/val_loss/0        

In [16]:
results.get_dataframe()

,loss,accuracy,timestamp,checkpoint_dir_name,should_checkpoint,done,training_iteration,trial_id,date,time_this_iter_s,...,iterations_since_restore,summary/epoch/0,summary/train_loss/0,summary/train_acc/0,summary/val_loss/0,summary/val_acc/0,config/train_loop_config/epochs,config/train_loop_config/batch_size,config/train_loop_config/lr,logdir
0,2.014310,0.899497,1712995376,checkpoint_000000,True,False,1,132f3_00000,2024-04-13_08-02-56,60.612070,...,1,1.0,3.231444,0.214018,2.014310,0.899497,1,32,0.000138,132f3_00000
1,2.067533,0.899497,1712995417,checkpoint_000000,True,False,1,132f3_00001,2024-04-13_08-03-37,39.797407,...,1,1.0,2.446435,0.367960,2.067533,0.899497,1,64,0.000251,132f3_00001


In [19]:
results.experiment_path

'/root/ray_results/tune_test'

In [23]:
best = results.get_best_result(metric='accuracy', mode='max')
print(best)

from pprint import pprint
pprint(best.config)

Result(
  metrics={'loss': 2.014309661728995, 'accuracy': 0.8994974874371859, 'summary': {'epoch': {0: 1.0}, 'train_loss': {0: 3.2314441479169407}, 'train_acc': {0: 0.21401752190237797}, 'val_loss': {0: 2.014309661728995}, 'val_acc': {0: 0.8994974874371859}}},
  path='/root/ray_results/tune_test/TorchTrainer_132f3_00000_0_batch_size=32,lr=0.0001_2024-04-13_08-01-46',
  filesystem='local',
  checkpoint=Checkpoint(filesystem=local, path=/root/ray_results/tune_test/TorchTrainer_132f3_00000_0_batch_size=32,lr=0.0001_2024-04-13_08-01-46/checkpoint_000000)
)
{'train_loop_config': {'batch_size': 32,
                       'epochs': 1,
                       'lr': 0.00013784989629017104}}


In [25]:
best.metrics_dataframe

,loss,accuracy,timestamp,checkpoint_dir_name,should_checkpoint,done,training_iteration,trial_id,date,time_this_iter_s,...,time_since_restore,iterations_since_restore,summary/epoch/0,summary/train_loss/0,summary/train_acc/0,summary/val_loss/0,summary/val_acc/0,config/train_loop_config/epochs,config/train_loop_config/batch_size,config/train_loop_config/lr
0,2.01431,0.899497,1712995376,checkpoint_000000,True,False,1,132f3_00000,2024-04-13_08-02-56,60.61207,...,60.61207,1,1.0,3.231444,0.214018,2.01431,0.899497,1,32,0.000138


In [ ]:
test_tuner = ciri_trainer.get_trainer("tune_test", as_trainable=True)
restored_tuner = tune.Tuner.restore("/root/ray_results/tune_test", trainable=test_tuner)

restored_tuner.get_results()

In [24]:
!du -sh /root/ray_results/tune_test/TorchTrainer_132f3_00000_0_batch_size=32,lr=0.0001_2024-04-13_08-01-46

32M	/root/ray_results/tune_test/TorchTrainer_132f3_00000_0_batch_size=32,lr=0.0001_2024-04-13_08-01-46


In [3]:
selected_model="resnet50"
root_base = os.path.join(project_folder, 'Incidents-subset')
root_augmented = os.path.join(project_folder, 'augmented_images')

data_folders=[root_base, root_augmented]

In [4]:
ciri_trainer = CIRI_trainer(model=selected_model,
                            data_folders=data_folders,
                            data_prop=0.8,
                            sample_indices=list(range(1000)), # remove this for full dataset
                            )

In [5]:
param_space = {
    "epochs": 1,
    "batch_size": tune.grid_search([32, 64])
}

In [6]:
cv_results = ciri_trainer.cross_validate(
    run_name="test_cv",
    config=param_space,
    outer_cv_k=5,
    inner_cv_k=3,
    tune_hyperparams=True,
    num_samples=2
)

Outer fold 0, inner fold 0 - number of samples: 533
Tuning hyperparameters for test_cv_outer_0_inner_0...
Defaulting to ASHA scheduler (no scheduler provided or not an instance of TrialScheduler)


/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-04-13 08:28:50,470	INFO worker.py:1752 -- Started a local Ray instance.
2024-04-13 08:28:52,501	INFO tune.py:263 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.
2024-04-13 08:28:52,509	INFO tune.py:622 -- [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949


+------------------------------------------------------------+
| Configuration for experiment     test_cv_outer_0_inner_0   |
+------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator     |
| Scheduler                        AsyncHyperBandScheduler   |
| Number of trials                 4                         |
+------------------------------------------------------------+

View detailed results here: /root/ray_results/test_cv_outer_0_inner_0
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-04-13_08-28-47_137033_12048/artifacts/2024-04-13_08-28-52/test_cv_outer_0_inner_0/driver_artifacts`

Trial status: 4 PENDING
Current time: 2024-04-13 08:28:52. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:T4)
+--------------------------------------------------------------+
| Trial name                 status       ...config/batch_size |
+--------

(TrainTrainable pid=12440) 2024-04-13 08:28:58.780842: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(TrainTrainable pid=12440) 2024-04-13 08:28:58.780900: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(TrainTrainable pid=12440) 2024-04-13 08:28:58.782521: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(TrainTrainable pid=12440) 2024-04-13 08:29:00.007179: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



Trial TorchTrainer_dc2ab_00000 started with configuration:
+----------------------------------------------------------------+
| Trial TorchTrainer_dc2ab_00000 config                          |
+----------------------------------------------------------------+
| train_loop_config/batch_size                                32 |
| train_loop_config/epochs                                     1 |
| train_loop_config/train_test_idx          ..., 989, 991, 993]) |
+----------------------------------------------------------------+


...


Trial status: 3 TERMINATED | 1 RUNNING
Current time: 2024-04-13 09:06:18. Total running time: 2min 0s
Logical resource usage: 0/2 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:T4)
+--------------------------------------------------------------------------------------------------------------------+
| Trial name                 status         ...config/batch_size     iter     total time (s)       loss     accuracy |
+--------------------------------------------------------------------------------------------------------------------+
| TorchTrainer_cece2_00003   RUNNING                          64                                                     |
| TorchTrainer_cece2_00000   TERMINATED                       32        1            32.253    0.86827      0.875472 |
| TorchTrainer_cece2_00001   TERMINATED                       64        1            33.5113   0.75866      0.875472 |
| TorchTrainer_cece2_00002   TERMINATED                       32        1            34.793    0.643266     

(RayTrainWorker pid=25157) 2024-04-13 09:06:18.400216: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(RayTrainWorker pid=25157) 2024-04-13 09:06:18.400280: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(RayTrainWorker pid=25157) 2024-04-13 09:06:18.401993: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(RayTrainWorker pid=25157) 2024-04-13 09:06:20.376460: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(RayTrainWorker pid=25157) Moving model to device: cuda:0
Epoch (test) 1:  67%|██████▋   | 6/9 [00:05<00:02,  1.08it/s]


Trial status: 3 TERMINATED | 1 RUNNING
Current time: 2024-04-13 09:06:48. Total running time: 2min 30s
Logical resource usage: 0/2 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:T4)
+--------------------------------------------------------------------------------------------------------------------+
| Trial name                 status         ...config/batch_size     iter     total time (s)       loss     accuracy |
+--------------------------------------------------------------------------------------------------------------------+
| TorchTrainer_cece2_00003   RUNNING                          64                                                     |
| TorchTrainer_cece2_00000   TERMINATED                       32        1            32.253    0.86827      0.875472 |
| TorchTrainer_cece2_00001   TERMINATED                       64        1            33.5113   0.75866      0.875472 |
| TorchTrainer_cece2_00002   TERMINATED                       32        1            34.793    0.643266     

Epoch (test) 1: 100%|██████████| 9/9 [00:07<00:00,  1.25it/s]
(RayTrainWorker pid=25157) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/test_cv_outer_4_inner_2/TorchTrainer_cece2_00003_3_batch_size=64_2024-04-13_09-04-17/checkpoint_000000)
2024-04-13 09:06:52,658	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).
2024-04-13 09:06:52,666	INFO tune.py:1016 -- Wrote the latest version of all result files and experiment state to '/root/ray_result


Trial TorchTrainer_cece2_00003 completed after 1 iterations at 2024-04-13 09:06:52. Total running time: 2min 35s
+--------------------------------------------------------------+
| Trial TorchTrainer_cece2_00003 result                        |
+--------------------------------------------------------------+
| checkpoint_dir_name                        checkpoint_000000 |
| time_this_iter_s                                     39.0155 |
| time_total_s                                         39.0155 |
| training_iteration                                         1 |
| accuracy                                             0.87547 |
| loss                                                 0.75197 |
| summary/epoch/0                                          1.0 |
| summary/train_acc/0                        0.774859287054409 |
| summary/train_loss/0                      0.8803168798194212 |
| summary/val_acc/0                         0.8754716981132076 |
| summary/val_loss/0                     

In [32]:
!du -sh /root/ray_results/test_cv_outer_0_inner_0

902M	/root/ray_results/test_cv_outer_0_inner_0


In [9]:
cv_results

defaultdict(dict,
            {0: {0: ResultGrid<[
                Result(
                  metrics={'loss': 0.5897108804848459, 'accuracy': 0.8726591760299626, 'summary': {'epoch': {0: 1.0}, 'train_loss': {0: 0.758504800936755}, 'train_acc': {0: 0.8030018761726079}, 'val_loss': {0: 0.5897108804848459}, 'val_acc': {0: 0.8726591760299626}}},
                  path='/root/ray_results/test_cv_outer_0_inner_0/TorchTrainer_dc2ab_00000_0_batch_size=32_2024-04-13_08-28-52',
                  filesystem='local',
                  checkpoint=Checkpoint(filesystem=local, path=/root/ray_results/test_cv_outer_0_inner_0/TorchTrainer_dc2ab_00000_0_batch_size=32_2024-04-13_08-28-52/checkpoint_000000)
                ),
                Result(
                  metrics={'loss': 0.7934163014094034, 'accuracy': 0.8689138576779026, 'summary': {'epoch': {0: 1.0}, 'train_loss': {0: 0.7732236367814681}, 'train_acc': {0: 0.8123827392120075}, 'val_loss': {0: 0.7934163014094034}, 'val_acc': {0: 0.868913857677

In [12]:
cv_results[0][0].get_best_result(metric='accuracy', mode='max').best_checkpoints

[(Checkpoint(filesystem=local, path=/root/ray_results/test_cv_outer_0_inner_0/TorchTrainer_dc2ab_00000_0_batch_size=32_2024-04-13_08-28-52/checkpoint_000000),
  {'loss': 0.5897108804848459,
   'accuracy': 0.8726591760299626,
   'summary': {'epoch': {0: 1.0},
    'train_loss': {0: 0.758504800936755},
    'train_acc': {0: 0.8030018761726079},
    'val_loss': {0: 0.5897108804848459},
    'val_acc': {0: 0.8726591760299626}},
   'timestamp': 1712996975,
   'checkpoint_dir_name': 'checkpoint_000000',
   'should_checkpoint': True,
   'done': False,
   'training_iteration': 1,
   'trial_id': 'dc2ab_00000',
   'date': '2024-04-13_08-29-36',
   'time_this_iter_s': 35.59644889831543,
   'time_total_s': 35.59644889831543,
   'pid': 12440,
   'hostname': '6235e01e8f24',
   'node_ip': '172.28.0.12',
   'config': {'train_loop_config': {'epochs': 1,
     'batch_size': 32,
     'train_test_idx': ([1,
       8,
       11,
       14,
       17,
       19,
       21,
       22,
       23,
       24,
     

In [13]:
cv_results = ciri_trainer.cross_validate(
    run_name="test_cv_nohp",
    config=None,
    outer_cv_k=5,
    inner_cv_k=0,
    tune_hyperparams=False
)

2024-04-13 09:22:13,065	INFO tune.py:622 -- [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949


Outer fold 0 - number of samples: 800

View detailed results here: /root/ray_results/test_cv_nohp_outer_0
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-04-13_08-28-47_137033_12048/artifacts/2024-04-13_09-22-13/test_cv_nohp_outer_0/driver_artifacts`


(TrainTrainable pid=29308) 2024-04-13 09:22:19.464428: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(TrainTrainable pid=29308) 2024-04-13 09:22:19.464488: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(TrainTrainable pid=29308) 2024-04-13 09:22:19.466285: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(TrainTrainable pid=29308) 2024-04-13 09:22:21.084252: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



Training started with configuration:
+---------------------------------------------------------+
| Training config                                         |
+---------------------------------------------------------+
| train_loop_config/data_folders     ...ugmented_images'] |
| train_loop_config/data_prop                         0.8 |
| train_loop_config/model                        resnet50 |
| train_loop_config/sample_indices   ...6, 997, 998, 999] |
| train_loop_config/train_test_idx   ..., 975, 994, 996]) |
+---------------------------------------------------------+


(TorchTrainer pid=29308) Started distributed worker processes: 
(TorchTrainer pid=29308) - (ip=172.28.0.12, pid=29377) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=29377) Setting up process group for: env:// [rank=0, world_size=1]
(RayTrainWorker pid=29377) [W Utils.hpp:133] Warning: Environment variable NCCL_ASYNC_ERROR_HANDLING is deprecated; use TORCH_NCCL_ASYNC_ERROR_HANDLING instead (function getCvarInt)
(RayTrainWorker pid=29377) 2024-04-13 09:22:26.585009: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(RayTrainWorker pid=29377) 2024-04-13 09:22:26.585061: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(RayTrainWorker pid=29377) 2024-04-13 09:22:26.586257: E external/local_xla/xla/stream_executor/c

...


Training finished iteration 9 at 2024-04-13 09:47:15. Total running time: 4min 40s
+--------------------------------------------+
| Training result                            |
+--------------------------------------------+
| checkpoint_dir_name      checkpoint_000008 |
| time_this_iter_s                  28.17659 |
| time_total_s                     268.01587 |
| training_iteration                       9 |
| accuracy                             0.825 |
| loss                               0.62874 |
| summary/epoch/0                        1.0 |
| summary/epoch/1                        2.0 |
| summary/epoch/2                        3.0 |
| summary/epoch/3                        4.0 |
| summary/epoch/4                        5.0 |
| summary/epoch/5                        6.0 |
| summary/epoch/6                        7.0 |
| summary/epoch/7                        8.0 |
| summary/epoch/8                        9.0 |
| summary/train_acc/0     0.8245614035087719 |
| summary/train_acc/1  

(RayTrainWorker pid=35041) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/test_cv_nohp_outer_4/TorchTrainer_28381_00000_0_2024-04-13_09-42-35/checkpoint_000008)
Epoch (test) 10: 100%|██████████| 7/7 [00:04<00:00,  1.49it/s]



Training finished iteration 10 at 2024-04-13 09:47:44. Total running time: 5min 9s
+--------------------------------------------+
| Training result                            |
+--------------------------------------------+
| checkpoint_dir_name      checkpoint_000009 |
| time_this_iter_s                  28.74922 |
| time_total_s                      296.7651 |
| training_iteration                      10 |
| accuracy                              0.83 |
| loss                               0.48426 |
| summary/epoch/0                        1.0 |
| summary/epoch/1                        2.0 |
| summary/epoch/2                        3.0 |
| summary/epoch/3                        4.0 |
| summary/epoch/4                        5.0 |
| summary/epoch/5                        6.0 |
| summary/epoch/6                        7.0 |
| summary/epoch/7                        8.0 |
| summary/epoch/8                        9.0 |
| summary/epoch/9                       10.0 |
| summary/train_acc/0  

(RayTrainWorker pid=35041) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/test_cv_nohp_outer_4/TorchTrainer_28381_00000_0_2024-04-13_09-42-35/checkpoint_000009)
2024-04-13 09:47:46,196	INFO tune.py:1016 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/test_cv_nohp_outer_4' in 0.0150s.



Training completed after 10 iterations at 2024-04-13 09:47:46. Total running time: 5min 11s



In [14]:
cv_results[0].get_best_checkpoint(metric='accuracy', mode='max').path

'/root/ray_results/test_cv_nohp_outer_0/TorchTrainer_4fd35_00000_0_2024-04-13_09-22-13/checkpoint_000000'

In [16]:
cv_results[0].path

'/root/ray_results/test_cv_nohp_outer_0/TorchTrainer_4fd35_00000_0_2024-04-13_09-22-13'

In [15]:
!du -sh /root/ray_results

21G	/root/ray_results
